## Installation Required

Before running this notebook, you need to install the Google Generative AI package. Run the cell below **only once**.

# Day 3 - Conversational AI with Gemini - aka Chatbot!

## What You'll Learn

In this notebook, you'll learn how to:
1. **Build a conversational chatbot** using Google's Gemini model
2. **Maintain conversation history** so the AI remembers context
3. **Use system instructions** to guide the AI's behavior and personality
4. **Stream responses** for a better user experience
5. **Create a web interface** with Gradio for easy interaction

## Key Concepts

### Conversation History
Unlike simple one-off questions, chatbots need to remember what was said before. We'll manage this history to create natural conversations.

### System Instructions
These are special instructions that guide how the AI behaves - like giving it a role or personality. Think of it as "behind the scenes" instructions the user doesn't see.

### Streaming
Instead of waiting for the complete response, we'll show words as they're generated - just like ChatGPT does!

In [ ]:
# Step 1: Import necessary libraries
# 
# os: to access environment variables
# dotenv: to load API keys from .env file
# google.genai: Google's Gemini AI library (newer SDK)
# gradio: for creating web interfaces

import os
from dotenv import load_dotenv
from google import genai
import gradio as gr

## Environment Setup

Before we can use Gemini, we need to:
1. Load our API key from the `.env` file
2. Verify the key is properly set
3. Configure the Gemini library to use our key

In [ ]:
# Step 2: Load and verify API key
#
# The load_dotenv() function reads your .env file and makes the variables available
# We print the first 8 characters to verify it's loaded (never print the full key!)

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"✓ Google API Key loaded successfully (begins with: {google_api_key[:8]})")
    # Create the Gemini client
    client = genai.Client(api_key=google_api_key)
else:
    print("✗ Google API Key not found - please check your .env file")

## Choose Your Model

Google offers several Gemini models:
- **gemini-2.0-flash-exp**: Latest experimental model, very fast and capable
- **gemini-1.5-flash**: Stable, fast, and cost-effective
- **gemini-1.5-pro**: Most capable but slower and more expensive

We'll use `gemini-1.5-flash` for a good balance of speed and quality.

In [ ]:
# Step 3: Initialize the model
#
# We're choosing gemini-2.0-flash-exp for:
# - Latest features
# - Fast responses
# - Good quality for most tasks

MODEL = 'gemini-2.0-flash-exp'

## System Instructions - The AI's Personality

System instructions are like giving the AI a "role" to play. They:
- Define how the AI should behave
- Set the tone and style of responses
- Provide context about what the AI is helping with
- Are invisible to the user

**Important**: System instructions apply to the ENTIRE conversation, not just one message.

In [ ]:
# Step 4: Define system instructions
#
# Start with a simple helpful assistant
# We'll make this more sophisticated later!

system_message = "You are a helpful assistant"

## Building the Chat Function

### How Gradio ChatInterface Works

Gradio's `ChatInterface` expects a function with this signature:
```python
def chat(message, history):
```

Where:
- **message**: The user's latest message (string)
- **history**: Past conversation in Gradio format (list of dictionaries)

### Gradio's History Format

Gradio passes history like this:
```python
[
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": "Hi! How can I help?"},
    {"role": "user", "content": "What's the weather?"},
    {"role": "assistant", "content": "I don't have weather data..."}
]
```

### Converting to Gemini Format

Gemini needs a slightly different format with "parts":
```python
[
    {"role": "user", "parts": "Hello!"},
    {"role": "model", "parts": "Hi! How can I help?"}  # Note: "model" not "assistant"
]
```

### Why We Need Streaming

Streaming means we show the response word-by-word as it's generated, rather than waiting for the complete answer. This:
- Feels more responsive
- Shows progress on long answers
- Matches the ChatGPT experience users expect

In [ ]:
# Step 5: Create the chat function
#
# This function:
# 1. Converts Gradio's history format to Gemini's format
# 2. Uses the Gemini client to generate responses
# 3. Generates a streaming response
# 4. Yields each chunk as it arrives (for gradual display)

def chat(message, history):
    # Convert Gradio format to Gemini format
    # Gradio uses: {"role": "assistant", "content": "text"}
    # Gemini needs: {"role": "model", "parts": [{"text": "text"}]} for model responses
    # and {"role": "user", "parts": [{"text": "text"}]} for user messages
    
    gemini_history = []
    for msg in history:
        role = "model" if msg["role"] == "assistant" else "user"
        gemini_history.append({
            "role": role,
            "parts": [{"text": msg["content"]}]
        })
    
    # Add the current user message
    gemini_history.append({
        "role": "user",
        "parts": [{"text": message}]
    })
    
    # Debugging: see what we're sending (helpful for learning!)
    print("\n=== Conversation History ===")
    print(f"System Instruction: {system_message}")
    print(f"Messages to Gemini:")
    for msg in gemini_history:
        print(f"  {msg['role']}: {msg['parts'][0]['text'][:50]}...")  # First 50 chars
    
    # Generate streaming response with system instruction
    response_stream = client.models.generate_content_stream(
        model=MODEL,
        contents=gemini_history,
        config={
            "system_instruction": system_message,
            "temperature": 0.7
        }
    )
    
    # Yield chunks as they arrive
    # This makes the text appear gradually in the interface
    response = ""
    for chunk in response_stream:
        if chunk.text:
            response += chunk.text
            yield response  # Each yield updates the display

## Launch Your First Chatbot!

The `gr.ChatInterface()` creates a complete chat UI with:
- Message input box
- Conversation history display
- Automatic handling of the conversation flow
- Mobile-responsive design

**Try it out**: 
- Ask it questions
- Have a conversation
- Notice how it remembers context from earlier messages!

In [ ]:
# Step 6: Launch the chatbot interface
#
# type="messages" tells Gradio to use the OpenAI message format
# (which we then convert to Gemini format in our function)

gr.ChatInterface(fn=chat, type="messages").launch()

## Real-World Example: Sales Assistant

Now let's build something practical - a sales assistant for a clothing store!

### Business Context
The store has a sale event:
- Hats: 60% off
- Most other items: 50% off

### Our Goal
Create an AI assistant that:
1. Helps customers find items
2. Subtly encourages them to look at sale items
3. Especially promotes hats (highest discount)
4. Stays helpful and not pushy

### The Power of System Instructions
Notice how we can give the AI context, examples, and behavioral guidelines all in the system message!

In [ ]:
# Step 7: Enhanced system message for a sales assistant
#
# Notice the techniques:
# 1. Clear role definition ("helpful assistant in a clothes store")
# 2. Specific business context (sale percentages)
# 3. Example of desired behavior
# 4. Gentle guidance on tone ("gently encourage")

system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.' \
Encourage the customer to buy hats if they are unsure what to get."

## Understanding the Chat Function (Simplified)

Since our system message changed but the chat logic is the same, we can reuse the same function.

**Key Insight**: By just changing the `system_message` variable, we completely changed the AI's behavior!

This is the power of prompt engineering.

In [ ]:
# Step 8: Launch with sales assistant personality
#
# Same code, different behavior!
# Try asking:
# - "What should I buy?"
# - "I need a new outfit"
# - "Tell me about your hats"

gr.ChatInterface(fn=chat, type="messages").launch()

## Iterative Improvement: Adding More Context

In real applications, you'll often need to refine the system message based on:
- User feedback
- Edge cases you discover
- Changing business requirements

Let's add handling for shoes (which aren't on sale).

In [ ]:
# Step 9: Iteratively improve the prompt
#
# We're adding to the existing system message
# This is common - start simple, then add edge cases

system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
# Step 10: Test the improved assistant
#
# Try asking: "Do you have shoes on sale?"

gr.ChatInterface(fn=chat, type="messages").launch()

## Advanced: Dynamic System Messages

Sometimes you want to change the system message based on what the user asks.

### Use Case: Out of Stock Items
If someone asks about belts (which you don't sell), add that information dynamically.

### Why This Matters
- You can't predict every question
- Some context is only relevant for certain queries
- Keeps system message focused and efficient

In [ ]:
# Step 11: Advanced chat function with dynamic system message
#
# This function modifies the system message based on the user's question
# Notice: we create a NEW variable so we don't permanently change system_message

def chat_dynamic(message, history):
    # Start with the base system message
    relevant_system_message = system_message
    
    # Add context if specific items are mentioned
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    # You could add more conditions:
    # if 'return' in message.lower():
    #     relevant_system_message += " Our return policy is 30 days with receipt."
    
    # Convert history to Gemini format
    gemini_history = []
    for msg in history:
        role = "model" if msg["role"] == "assistant" else "user"
        gemini_history.append({
            "role": role,
            "parts": [{"text": msg["content"]}]
        })
    
    gemini_history.append({
        "role": "user",
        "parts": [{"text": message}]
    })
    
    # Generate streaming response with dynamic system instruction
    response_stream = client.models.generate_content_stream(
        model=MODEL,
        contents=gemini_history,
        config={
            "system_instruction": relevant_system_message,
            "temperature": 0.7
        }
    )
    
    response = ""
    for chunk in response_stream:
        if chunk.text:
            response += chunk.text
            yield response

In [ ]:
# Step 12: Test dynamic system messages
#
# Try:
# - "Do you have belts?" (should trigger the belt message)
# - "What's on sale?" (won't trigger it)

gr.ChatInterface(fn=chat_dynamic, type="messages").launch()

## Business Applications

### Why This Matters

Conversational AI assistants are transforming businesses:

1. **Customer Service**: 24/7 support with context awareness
2. **Sales**: Personalized recommendations based on conversation
3. **Education**: Tutoring that adapts to student needs
4. **Healthcare**: Patient intake and triage
5. **HR**: Employee onboarding and FAQ

### What You've Learned

1. ✅ Build conversational interfaces with Gradio
2. ✅ Manage conversation history for context
3. ✅ Use system instructions to guide AI behavior
4. ✅ Stream responses for better UX
5. ✅ Dynamically adapt behavior based on user input
6. ✅ Apply to real business scenarios

### Your Turn!

Think about your business or a business you know:
- What repetitive questions do customers ask?
- What information do customers need?
- How could an AI assistant help?

Try building your own chatbot with:
- Custom system instructions for your use case
- Relevant business context
- Dynamic messages for edge cases

### Next Steps

To make this production-ready, consider:
1. **Error handling**: What if the API fails?
2. **Rate limiting**: Prevent abuse
3. **Logging**: Track conversations for improvement
4. **Testing**: Ensure consistent behavior
5. **Integration**: Connect to your database/CRM

## Key Differences: Gemini vs OpenAI

| Aspect | Gemini | OpenAI |
|--------|---------|--------|
| **History format** | `{"role": "model", "parts": "..."}` | `{"role": "assistant", "content": "..."}` |
| **System message** | Via `system_instruction` parameter | As first message with role "system" |
| **Streaming** | `.generate_content(stream=True)` | `.create(stream=True)` |
| **Response access** | `chunk.text` | `chunk.choices[0].delta.content` |
| **Model initialization** | `GenerativeModel()` for each chat | One `OpenAI()` client reused |

### Why These Differences?

Different AI providers have different APIs because:
- They evolved independently
- Different underlying architectures
- Different design philosophies

**Good news**: The concepts are the same! Once you understand one, adapting to others is straightforward.

---

## Congratulations! 🎉

You've built a sophisticated conversational AI system using Gemini. You now understand:
- How chat interfaces work
- How to manage conversation context
- How to guide AI behavior with prompts
- How to create practical business applications

This is a foundational skill in modern AI development!

In [ ]:
# Experiment zone - build your own chatbot!
# 
# Try creating a chatbot for:
# - A restaurant (taking orders, dietary restrictions)
# - A tech support desk (troubleshooting common issues)
# - A fitness coach (workout advice, motivation)
# - A language tutor (practice conversations)
# 
# Start by defining your system_message here:

my_system_message = """Your custom system instructions here!"""

def my_chat(message, history):
    gemini_history = []
    for msg in history:
        role = "model" if msg["role"] == "assistant" else "user"
        gemini_history.append({
            "role": role,
            "parts": [{"text": msg["content"]}]
        })
    
    gemini_history.append({
        "role": "user",
        "parts": [{"text": message}]
    })
    
    response_stream = client.models.generate_content_stream(
        model=MODEL,
        contents=gemini_history,
        config={
            "system_instruction": my_system_message,
            "temperature": 0.7
        }
    )
    
    response = ""
    for chunk in response_stream:
        if chunk.text:
            response += chunk.text
            yield response

# Uncomment to launch your custom chatbot:
# gr.ChatInterface(fn=my_chat, type="messages").launch()